In [161]:
import cvxpy as cp
import numpy as np

def Solver(data):
    n, w, v, W, G = data
    x = cp.Variable(n, boolean=True)
    
    cons = []
    cons.append(x @ w <= W)
    #cons.append(x @ v >= G)
    obj = x @ v
    
    problem = cp.Problem(cp.Maximize(obj), cons)
    problem.solve(solver="CBC")
    print('Status: ', problem.status)
    print('Weight:', (x @ w).value)
    return problem.value, x.value

def GetData(filename):
    with open(filename, 'r') as file:
        file.readline()
        n = int(file.readline()[1:])
        W = int(file.readline()[1:])
        G = int(file.readline()[1:])
        file.readline()
        w, v = [], []
        ans = []
        for line in file.readlines():
            line = line.strip()
            line = list(map(int, line.split(',')))
            v.append(line[1])
            w.append(line[2])
            ans.append(line[3])
            if len(w) >= n:
                break
    return n, w, v, W, G, ans

def Knapsack_LP(filename):
    n, w, v, W, G, check_answer = GetData(filename)
    pr, x = Solver((n, w, v, W, G))    
    print('Ответ:', pr)
    print('Равенство ответов', list(x) == check_answer)
    
    w_check, v_check = 0, 0
    for i in range(n):
        if check_answer[i] == 1:
            v_check += v[i]
    print('Равенство ценности', pr == v_check)
    return x

In [162]:
filename = 'knapPI_3_50_10000.csv'
ans0 = Knapsack_LP(filename)
ans0

Status:  optimal
Weight: 9862.0
Ответ: 20862.0
Равенство ответов False
Равенство ценности True


array([1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.])

In [163]:
n, w, v, W, G, ans = GetData(filename)

temp = list(zip(w, v, list(range(n))))
temp.sort(key=lambda x: x[0])
w, v, ind = [], [], []
for wi, vi, xi in temp:
    w.append(wi)
    v.append(vi)
    ind.append(xi)

In [164]:
def Knapsack_Dynamic(k, s):
    if A[k][s] == 0:
        return 0
    if A[k-1][s] == A[k][s]:
        return Knapsack_Dynamic(k-1, s)
    else:
        ans2.append(k-1)
        check_weight.append(w[k-1])
        return Knapsack_Dynamic(k-1, s - w[k-1])

A = np.ones((n+1,W+1))
for i in range(n+1):
    A[i][0] = 0
    
for i in range(W+1):
    A[0][i] = 0
    
for k in range(n):
    for s in range(1, W+1):
        if s >= w[k]:
            A[k+1][s] = max(A[k][s], A[k][s - w[k]] + v[k])
        else:
            A[k+1][s] = A[k][s]

print("Ответ:", A[n][W])

ans2 = []
check_weight = []
Knapsack_Dynamic(n, W)
print("Суммарный вес:", sum(check_weight))
ans2

Ответ: 20862.0
Суммарный вес: 9862


[11, 10, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [165]:
#Сравнение результатов динамического программирования и LP-solvera
#Совпадают ли они?
check = [0]*n
for i in ans2:
    check[ind[i]] = 1
print('Равенство ответов', check == list(map(round, ans0)))

Равенство ответов True


Стоит упомянуть, что решение солвером гораздо выгоднее, так как менее требователен к пользователю и компьютеру. 

На больших данных решение динамический программированием у меня не сработало, так как на матрицу размера 10000х59534113 не хватает памяти

Тестирование программы проводилось на следующих датасетах:

1. knapPI_11_10000_1000.csv
2. knapPI_16_10000_1000.csv 
3. knapPI_1_10000_10000.csv
4. knapPI_2_10000_100000.csv
5. knapPI_2_50_1000.csv
6. knapPI_3_50_10000.csv
7. knapPI_4_10000_1000000.csv
8. knapPI_6_10000_10000.csv